# Motoparts

Motoparts is an imaginary company that sells motorcycle parts, and they've asked for some help in analysing their sales data!

They operate three warehouses in Australia, selling both retail and wholesale. They offer a variety of parts and accept credit cards, cash, and bank transfer as payment methods. However, each payment type incurs a different fee.

The board of directors wants to gain a better understanding of wholesale revenue by product line, and how this varies month-to-month and across warehouses. You have been tasked with calculating net revenue for each product line and grouping results by month and warehouse. The results should be filtered so that only `"Wholesale"` orders are included.

They have provided you with access to their database, which contains the following table called `sales`:

## Schema
| Column | Data type | Description |
|--------|-----------|-------------|
| `order_number` | `VARCHAR` | Unique order number. |
| `date` | `DATE` | Date of the order, from June to August 2021. |
| `warehouse` | `VARCHAR` | The warehouse that the order was made from&mdash; `North`, `Central`, or `West`. |
| `client_type` | `VARCHAR` | Whether the order was `Retail` or `Wholesale`. |
| `product_line` | `VARCHAR` | Type of product ordered. |
| `quantity` | `INT` | Number of products ordered. | 
| `unit_price` | `FLOAT` | Price per product (dollars). |
| `total` | `FLOAT` | Total price of the order (dollars). |
| `payment` | `VARCHAR` | Payment method&mdash;`Credit card`, `Transfer`, or `Cash`. |
| `payment_fee` | `FLOAT` | Percentage of `total` charged as a result of the `payment` method. |


## Requirements

They have requested the query output be presented in the following format:

| `product_line` | `month` | `warehouse` |	`net_revenue` |
|----------------|-----------|----------------------------|--------------|
| product_one | --- | --- | --- |
| product_one | --- | --- | --- |
| product_one | --- | --- | --- |
| product_one | --- | --- | --- |
| product_one | --- | --- | --- |
| product_one | --- | --- | --- |
| product_two | --- | --- | --- |
| ... | ... | ... | ... |

# Solution

Hi! Tom here, let's get started.

After assessing the schema for the _sales_ table, and considering the expected output for this query, it is clear that we are only interested in the following columns:

- product_line (unchanged)
- date (converted to mmmm) AS month
- warehouse (unchanged)
- total & payment fee (SUM(total) - SUM(payment_fee)) AS net_revenue


We are only interested in records where:

- client_type is "Wholesale"


The output then needs to be ordered by product_line, month and net_revenue.

## Understanding the Data

Lets start simple and run an **ordered select all**

In [23]:
SELECT * FROM public.sales

ORDER BY product_line, date, total DESC

,order_number,date,warehouse,client_type,product_line,quantity,unit_price,total,payment,payment_fee
0,C3,2021-06-01 00:00:00+00:00,Central,Retail,Braking system,10,22.44,224.38,Credit card,0.03
1,N1,2021-06-01 00:00:00+00:00,North,Retail,Braking system,9,19.29,173.61,Cash,0.00
2,C7,2021-06-02 00:00:00+00:00,Central,Retail,Braking system,9,15.10,135.92,Credit card,0.03
3,W3,2021-06-02 00:00:00+00:00,West,Retail,Braking system,2,24.97,49.94,Credit card,0.03
4,C14,2021-06-04 00:00:00+00:00,Central,Retail,Braking system,4,19.78,79.14,Credit card,0.03
...,...,...,...,...,...,...,...,...,...,...
995,W171,2021-08-24 00:00:00+00:00,West,Retail,Suspension & traction,2,39.04,78.08,Cash,0.00
996,N325,2021-08-25 00:00:00+00:00,North,Wholesale,Suspension & traction,8,32.39,259.12,Transfer,0.01
997,N331,2021-08-26 00:00:00+00:00,North,Retail,Suspension & traction,7,29.34,205.36,Credit card,0.03
998,N335,2021-08-27 00:00:00+00:00,North,Wholesale,Suspension & traction,12,37.07,444.84,Transfer,0.01


Interesting...

Some observations on the selected data:

- There are exactly 1000 rows.
- _order_number_ appears to be comprised of the first letter of _warehouse_ followed by a unique id number.
- _date_ appears to be stored as datetime
- The "Credit card" payment method incurs a 3c fee, "Transfer" incurs 1c and "Cash" incurs no fee.

## Cleaning

lets cull the records and columns we don't need and work on our custom columns, _month_ and _net_revenue_, before we start grouping.

In [24]:
SELECT
product_line,
to_char(date, 'Month') AS month,
warehouse,
total - payment_fee AS net_revenue

FROM public.sales

WHERE client_type = 'Wholesale'

ORDER BY product_line, "month", "net_revenue" DESC

,product_line,month,warehouse,net_revenue
0,Braking system,August,West,893.81
1,Braking system,August,Central,820.32
2,Braking system,August,West,806.59
3,Braking system,August,North,593.75
4,Braking system,August,Central,567.92
...,...,...,...,...
220,Suspension & traction,June,North,406.59
221,Suspension & traction,June,Central,343.79
222,Suspension & traction,June,North,323.87
223,Suspension & traction,June,West,159.06


Great!

We have the desired records and columns using the requested schema, ordered in the correct sequence.

## Final Output: Grouping & Aggregating

Time to GROUP BY and aggregate our _net_revenue_. 

Our output should then be ready to submit to the board of directors.

In [25]:
SELECT
product_line,
to_char(date, 'Month') AS month,
warehouse,
SUM(total) - SUM(payment_fee) AS net_revenue

FROM public.sales

WHERE client_type = 'Wholesale'

GROUP BY product_line, "month", warehouse
ORDER BY product_line, "month", "net_revenue" DESC


,product_line,month,warehouse,net_revenue
0,Braking system,August,Central,3039.41
1,Braking system,August,West,2500.67
2,Braking system,August,North,1770.84
3,Braking system,July,Central,3778.65
4,Braking system,July,West,3060.93
5,Braking system,July,North,2594.44
6,Braking system,June,Central,3684.89
7,Braking system,June,North,1487.77
8,Braking system,June,West,1212.75
9,Electrical system,August,North,4721.12


## Concluding Notes

We now have an indication of the wholesale product line performance by month and warehouse in the requested format, however I have some concerns regarding the readability and usefulness of this output. 

It is quite difficult to compare the performance of product lines **against eachother** using a common dimension such as month or warehouse. 

For example; has a particular product line improved or weakened over time? Are there product lines that excel in some warehouses but not others? 

Having understood the objectives of the board of directors, it may be worth going above and beyond their original request to visualise wholesale product line performance. These visualisations will provide greater context and supplement their strategic analysis. 